# 📸 Detección de Matrículas con YOLO y EasyOCR
Este notebook detecta matrículas en imágenes usando YOLOv8 y extrae su texto con EasyOCR. Además, muestra las imágenes procesadas con los resultados obtenidos.

In [ ]:

import cv2
import easyocr
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Cargar modelo YOLO para detección de matrículas
model = YOLO("license_plate_detector.pt")
    

## 📌 Funciones para detección y extracción de texto

In [ ]:

def detect_license_plate(image_path):
    """
    Detecta la matrícula en una imagen usando YOLOv8.
    """
    results = model(image_path)
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    for result in results:
        for box in result.boxes.xyxy:
            x1, y1, x2, y2 = map(int, box[:4])
            plate_img = image[y1:y2, x1:x2]
            return plate_img, (x1, y1, x2, y2), image_rgb
    
    print("No se detectó ninguna matrícula.")
    return None, None, image_rgb

def extract_text_from_plate(plate_img):
    """
    Usa EasyOCR para extraer el texto de la matrícula.
    """
    reader = easyocr.Reader(['en'])
    results = reader.readtext(plate_img, detail=0)
    return " ".join(results).strip()
    

## 🔍 Cargar la imagen y realizar detección

In [ ]:

# Ruta de la imagen a analizar
image_path = "imagen4.jpg"  # Reemplaza con la ruta de tu imagen

# Detectar matrícula
plate_img, bbox, original_image = detect_license_plate(image_path)

if plate_img is not None:
    # Extraer texto
    plate_text = extract_text_from_plate(plate_img)
    
    # Dibujar el rectángulo en la imagen original
    x1, y1, x2, y2 = bbox
    cv2.rectangle(original_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # Mostrar imágenes
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))
    ax[0].imshow(original_image)
    ax[0].set_title("Imagen Original con Detección")
    ax[0].axis("off")
    
    ax[1].imshow(cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB))
    ax[1].set_title("Matrícula Detectada")
    ax[1].axis("off")
    
    ax[2].text(0.5, 0.5, plate_text, fontsize=20, ha='center', va='center', 
                bbox=dict(facecolor='yellow', alpha=0.5))
    ax[2].set_title("Texto Detectado")
    ax[2].axis("off")
    
    plt.show()
else:
    print("No se pudo detectar la matrícula.")
    